In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from matplotlib.ticker import ScalarFormatter
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [30]:
import os

if os.name == 'posix':
    plt.rc("font", family="AppleGothic")
else:
    plt.rc("font", family="Malgun Gothic")

In [31]:
df = pd.read_csv('C:/Users/Home/Desktop/Data/2차/CardData_new.csv', encoding = 'cp949')

In [32]:
df2 = df.copy()

In [33]:
d = {
    '21.Active_연속' : 0,
    '22.Active_간헐' : 1,
    '34.휴면_9M+' : 2,
    '32.휴면_6M-' : 2,
    '33.휴면_9M-' : 2,
    '31.휴면_3M' : 2
}
df2['고객구분'] = df2['고객구분'].map(d)

# RFM 0,1 분류

In [34]:
df2['Recency'] = (df[['이용후경과월_신용','이용후경과월_체크', '이용후경과월_카드론']]).min(axis= 1)

In [35]:
df2['Recency_2'] = 0
const = (df2[['이용건수_신용_B0M', '이용건수_체크_B0M', '이용건수_카드론_B0M']].sum(axis = 1)).quantile(0.40)

df2.loc[(df2['Recency'] == 0) & (df2[['이용건수_신용_B0M', '이용건수_체크_B0M', '이용건수_카드론_B0M']].sum(axis = 1) >= const), ['Recency_2']] = 1

In [36]:
df2['Recency_2'].value_counts()

1    183224
0    116776
Name: Recency_2, dtype: int64

In [37]:
df2['Frequency'] = df2['이용건수_신용_R12M'] + df2['이용건수_체크_R12M'] + df2['이용건수_카드론_R12M']

In [38]:
threshold = df2['Frequency'].quantile(0.9)
df2['Frequency_2'] = df2['Frequency'].apply(lambda x: 1 if x >= threshold else 0)

In [39]:
df2['Monetary'] = df2['이용금액_신용_R12M'] + df2['이용금액_체크_R12M'] + df2['이용금액_카드론_R12M'] + df2['이용금액_업종기준']

In [40]:
threshold = df2['Monetary'].quantile(0.9)
df2['Monetary_2'] = df2['Monetary'].apply(lambda x: 1 if x >= threshold else 0)

In [41]:
df2['고객분류_RFM_2'] = '임시'

df2.loc[(df2['Recency_2'] == 1) & (df2['Frequency_2'] == 1) & (df2['Monetary_2'] == 1), ['고객분류_RFM_2']] = 'VIP고객'
df2.loc[(df2['Recency_2'] == 1) & (df2['Frequency_2'] == 1) & (df2['Monetary_2'] == 0), ['고객분류_RFM_2']] = '충성고객'
df2.loc[(df2['Recency_2'] == 1) & (df2['Frequency_2'] == 0) & (df2['Monetary_2'] == 1), ['고객분류_RFM_2']] = '잠재VIP고객'
df2.loc[(df2['Recency_2'] == 1) & (df2['Frequency_2'] == 0) & (df2['Monetary_2'] == 0), ['고객분류_RFM_2']] = '잠재충성고객'
df2.loc[(df2['Recency_2'] == 0) & (df2['Frequency_2'] == 1) & (df2['Monetary_2'] == 1), ['고객분류_RFM_2']] = '놓치면안될고객'
df2.loc[(df2['Recency_2'] == 0) & (df2['Frequency_2'] == 1) & (df2['Monetary_2'] == 0), ['고객분류_RFM_2']] = '관심필요고객'
df2.loc[(df2['Recency_2'] == 0) & (df2['Frequency_2'] == 0) & (df2['Monetary_2'] == 1), ['고객분류_RFM_2']] = '이탈우려고객'
df2.loc[(df2['Recency_2'] == 0) & (df2['Frequency_2'] == 0) & (df2['Monetary_2'] == 0), ['고객분류_RFM_2']] = '겨울잠고객'

In [42]:
df2['고객분류_RFM_2'].value_counts()

잠재충성고객     138529
겨울잠고객      114367
충성고객        16767
잠재VIP고객     15001
VIP고객       12927
이탈우려고객       2000
관심필요고객        337
놓치면안될고객        72
Name: 고객분류_RFM_2, dtype: int64

In [43]:
df2['고객분류_RFM_2'].value_counts()

잠재충성고객     138529
겨울잠고객      114367
충성고객        16767
잠재VIP고객     15001
VIP고객       12927
이탈우려고객       2000
관심필요고객        337
놓치면안될고객        72
Name: 고객분류_RFM_2, dtype: int64

# RFM 5개 값 분류

In [44]:
df2['Recency_5'] = (df[['이용후경과월_신용','이용후경과월_체크', '이용후경과월_카드론']]).min(axis= 1)

In [45]:
df2['Recency_5'].value_counts().sort_index()

0     243637
1       8549
2       5339
3       4252
4       3121
5       2768
6       2468
7       2273
8       1903
9       1721
10      1644
11      1539
12     20786
Name: Recency_5, dtype: int64

In [46]:
df2['Recency_5'] = df2['Recency'].map(lambda x : {0 : 5, 1 : 4, 2 :4, 3 : 4, 4: 3, 5:3, 6:3, 7:2, 8:2, 9:2, 10:1, 11:1, 12:1}[x])

In [47]:
df2['Frequency_5'] = pd.cut(df2['Frequency'], bins = [df2['Frequency'].quantile(0), df2['Frequency'].quantile(0.2), df2['Frequency'].quantile(0.4),
                                 df2['Frequency'].quantile(0.6), df2['Frequency'].quantile(0.8), df2['Frequency'].quantile(1)],
      labels = [1, 2, 3, 4, 5], include_lowest = True)

In [48]:
df2['Monetary_5'] = pd.cut(df2['Monetary'], bins = [df2['Monetary'].quantile(0), df2['Monetary'].quantile(0.2), df2['Monetary'].quantile(0.4),
                                 df2['Monetary'].quantile(0.6), df2['Monetary'].quantile(0.8), df2['Monetary'].quantile(1)],
      labels = [1, 2, 3, 4, 5], include_lowest = True)